In [1]:
# =========================================
# 1) 설치
# =========================================
!pip install -q openai gradio feedparser pandas python-dateutil


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.3 MB/s eta 0:00:00


In [ ]:
# =========================================
# 2) 임포트
# =========================================
import os, time, textwrap, feedparser, pandas as pd
from datetime import datetime, timezone
from dateutil import parser as dtparse
import gradio as gr
from openai import OpenAI

import os
from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
API_KEY = os.getenv("API_KEY")

# =========================================
# 3) OpenAI 클라이언트 설정
# =========================================
client = OpenAI(api_key=API_KEY)  # <- 본인 API 키 입력


In [4]:



# =========================================
# 4) RSS 소스 정의 (원하면 추가 가능)
# =========================================
GOOGLE_NEWS_SEARCH = "https://news.google.com/rss/search?q={query}&hl=ko&gl=KR&ceid=KR:ko"
RSS_SOURCES = {
    "Google News 검색": GOOGLE_NEWS_SEARCH,                              # 검색형
    "연합뉴스 속보": "https://www.yna.co.kr/rss/headlines.xml",          # 국내 대표 통신사
    "KBS 주요뉴스": "http://world.kbs.co.kr/rss/news.xml?lang=k",        # KBS
    "한겨레 최신": "https://www.hani.co.kr/rss/",                        # 한겨레
    "JTBC 전체": "https://fs.jtbc.co.kr/RSS/newsflash.xml",              # JTBC
}

# =========================================
# 5) 유틸: RSS 가져와서 DataFrame으로 정리
# =========================================
def fetch_news(source_name, query, max_items=10):
    if source_name == "Google News 검색":
        if not query.strip():
            raise ValueError("검색어를 입력해 주세요.")
        url = GOOGLE_NEWS_SEARCH.format(query=query.replace(" ", "+"))
    else:
        url = RSS_SOURCES[source_name]

    d = feedparser.parse(url)
    rows = []
    for e in d.entries[: max_items * 2]:  # 여유로 받아서 중복/파싱 제외
        title = getattr(e, "title", "").strip()
        link = getattr(e, "link", "").strip()
        summary = getattr(e, "summary", "").strip()
        # 날짜 파싱 (없을 수도 있음)
        published = None
        if hasattr(e, "published"):
            try:
                published = dtparse.parse(e.published).astimezone(timezone.utc)
            except Exception:
                published = None
        rows.append({
            "제목": title,
            "요약": summary,
            "링크": link,
            "발행시각(UTC)": published.isoformat() if published else ""
        })

    # DataFrame 정리
    df = pd.DataFrame(rows)
    # 중복 제거
    if not df.empty:
        df.drop_duplicates(subset=["제목"], inplace=True)
        # 최근순 정렬
        def _to_ts(x):
            try:
                return dtparse.parse(x)
            except Exception:
                return datetime(1970,1,1, tzinfo=timezone.utc)
        df["ts"] = df["발행시각(UTC)"].apply(_to_ts)
        df = df.sort_values("ts", ascending=False).head(max_items).drop(columns=["ts"]).reset_index(drop=True)
    return df

# =========================================
# 6) GPT 요약 (배치 요약: 헤드라인/요약들로 묶어서 요약)
#    - 환각 억제를 위해 "제공된 기사 목록만" 근거로 답변하도록 지시
# =========================================
def summarize_with_gpt(df, style="요약"):
    if df is None or df.empty:
        return " 요약할 뉴스가 없습니다. 먼저 뉴스를 불러와 주세요."

    # 컨텍스트 구성 (기사 인덱스+제목+요약만)
    items = []
    for i, row in df.iterrows():
        title = row.get("제목", "")
        summary = row.get("요약", "")
        link = row.get("링크", "")
        items.append(f"[{i+1}] {title}\n- {summary}\n- {link}")

    context = "\n\n".join(items[:15])  # 과한 길이 방지

    system = (
        "너는 뉴스 에디터다. 반드시 제공된 기사 목록만 근거로 한국어로 답하라. "
        "외부 추측/확장 금지. 핵심 포인트를 간결한 불릿으로 요약하고, 각 포인트에 관련 기사 번호를 괄호로 붙여라."
    )
    user = f"""다음은 최신 뉴스 기사 목록이다. '{style}' 관점으로 5~8줄로 요약해줘.
기사 목록:
{context}"""

    try:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role":"system","content":system},
                {"role":"user","content":user}
            ],
            temperature=0.2,
            seed=7
        )
        return resp.choices[0].message.content
    except Exception as e:
        return f" 요약 중 오류: {e}"

# =========================================
# 7) GPT 질의응답 (컨텍스트 = 표에 담긴 기사들)
#    - 답변에 기사 번호 인용하도록 요구
# =========================================
def qa_with_gpt(df, question):
    if not question.strip():
        return " 질문을 입력해 주세요."
    if df is None or df.empty:
        return " 먼저 뉴스를 불러오세요."

    items = []
    for i, row in df.iterrows():
        title = row.get("제목", "")
        summary = row.get("요약", "")
        link = row.get("링크", "")
        items.append(f"[{i+1}] {title}\n- {summary}\n- {link}")

    context = "\n\n".join(items[:20])

    system = (
        "너는 뉴스 분석가다. 반드시 제공된 기사 목록만 근거로 한국어로 답하라. "
        "확실히 근거가 없으면 모른다고 답하라. 관련 기사 번호를 괄호로 함께 표기하라."
    )
    user = f"""기사 목록:
{context}

질문: {question}
- 답변은 3~6문장으로.
- 확실한 근거가 없으면 '모름'이라고 말해라.
- 관련 기사 번호를 (예: [2][5])처럼 인용해라.
"""

    try:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role":"system","content":system},
                {"role":"user","content":user}
            ],
            temperature=0.2
        )
        return resp.choices[0].message.content
    except Exception as e:
        return f" 질의응답 오류: {e}"

# =========================================
# 8) Gradio UI
# =========================================
with gr.Blocks(title="뉴스 비서 챗봇") as demo:
    gr.Markdown("##  뉴스 비서 챗봇\n검색어 또는 소스를 선택해 최신 헤드라인을 불러오고, GPT로 요약/질문 응답을 받아보세요.")

    with gr.Row():
        source = gr.Dropdown(list(RSS_SOURCES.keys()), value="Google News 검색", label="뉴스 소스")
        query = gr.Textbox(label="검색어 (Google News 검색에서만 필요)", placeholder="예: 반도체, 테슬라, 원/달러 환율")
        n_items = gr.Slider(5, 20, value=10, step=1, label="기사 개수")
    load_btn = gr.Button(" 뉴스 불러오기", variant="primary")


    df_out = gr.Dataframe(
        headers=["제목","요약","링크","발행시각(UTC)"],
        label="헤드라인 표",
        wrap=True,
        interactive=False
    )

    with gr.Row():
        style = gr.Radio(["요약","시장영향 관점","기술 관점","정책/규제 관점"], value="요약", label="요약 스타일")
        summarize_btn = gr.Button(" GPT 요약")
    summary_md = gr.Markdown()

    with gr.Row():
        question = gr.Textbox(label="질문 (예: 가장 많이 보도된 이슈는? 주요 기업은?)")
        qa_btn = gr.Button(" 질의응답")
    answer_md = gr.Markdown()

    # 상태에 df를 저장하기 위한 State
    df_state = gr.State(pd.DataFrame())

    # 이벤트: 뉴스 불러오기
    def on_load(source_name, query_text, k):
        try:
            df = fetch_news(source_name, query_text or "", int(k))
            return df, df  # df_state, df_out
        except Exception as e:
            return pd.DataFrame(), gr.update(value=None, label=f"오류: {e}")

    load_btn.click(on_load, inputs=[source, query, n_items], outputs=[df_state, df_out])

    # 이벤트: 요약
    summarize_btn.click(lambda d, s: summarize_with_gpt(d, s), inputs=[df_state, style], outputs=summary_md)

    # 이벤트: Q&A
    qa_btn.click(lambda d, q: qa_with_gpt(d, q), inputs=[df_state, question], outputs=answer_md)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://39260304b3f9d31a3e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
